In [1]:
#NB!https://github.com/estnltk/estnltk/blob/master/README.md 
import urllib.request, os
from bs4 import BeautifulSoup
import pandas as pd
from pathlib import Path
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
import shutil
import os

In [2]:
#Katsetatud 3 tabeliga:
#Pohitabel_stopid_eemald_lower
#Pohitabel_ainult_reso
#Pohitabel_ilma_resota
data = pd.read_csv("Pohitabel_stopid_eemald_lower.csv")
data

,Tag,Lõigu sisu,Pealkiri1,Kohtuasja_nr,Kohtuasja_ID,algus_lopp,stop_eemaldatud
0,C01PointnumeroteAltN,"27 Kõigepealt väärib märkimist, et väljak...",NaN,C‑416/20 PPU,0,algus,27 väärib märkimist väljakujunenud kohtuprakti...
1,C01PointnumeroteAltN,28 Käesoleval juhul nähtub eelotsusetaotl...,NaN,C‑416/20 PPU,0,algus,28 käesoleval nähtub eelotsusetaotlusest eelot...
2,C01PointnumeroteAltN,29 Eelotsusetaotluse esitanud kohtu hinna...,NaN,C‑416/20 PPU,0,algus,29 eelotsusetaotluse esitanud kohtu hinnangul ...
3,C01PointnumeroteAltN,30 Neil asjaoludel tuleb mõista eelotsuse...,NaN,C‑416/20 PPU,0,algus,30 asjaoludel mõista eelotsusetaotluse esitanu...
4,C01PointnumeroteAltN,"31 Sellega seoses tuleb märkida, et nagu ...",NaN,C‑416/20 PPU,0,algus,31 seoses märkida nähtub raamotsuse 2002/584 a...
...,...,...,...,...,...,...,...
317516,C01PointnumeroteAltN,95 Kõnesoleva hüvitise hindamisel tuleks ...,NaN,F‑1/05,23801,algus,95 kõnesoleva hüvitise hindamisel arvesse võtt...
317517,C48DispositifIntroduction,Esitatud põhjendustest lähtudes,Resolutsioon,F‑1/05,23801,algus,esitatud põhjendustest lähtudes
317518,C08Dispositif,1. Tühistada Euroopa Koolitusfondi 25. ju...,Resolutsioon,F‑1/05,23801,algus,1. tühistada euroopa koolitusfondi 25. juuni 2...
317519,C08Dispositif,2. Pooled teatavad Avaliku Teenistuse Koh...,Resolutsioon,F‑1/05,23801,algus,2. teatavad avaliku teenistuse kohtule kolme k...


In [3]:
#paneme listi stop_eemaldatud
puhas=data[~pd.isnull(data.stop_eemaldatud)] #eemaldame enne ära null väärtused
column_list=puhas['stop_eemaldatud'].to_list()

In [4]:
# https://colab.research.google.com/drive/1Uh-w_oCllFMaUQOBZVL6xsdSyOfXBTxK?usp=sharing#scrollTo=AUORCu_ztPwG 
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
vectorizer = CountVectorizer()

In [5]:
#Meetod teksti sõnestamiseks ja lemmatiseerimiseks estnltk abil
from estnltk import Text
def lemmatize_with_estnltk(input_string):
    lemmas = []
    text = Text(input_string).tag_layer()
    for word in text.morph_analysis:
        lemmas.append(word.lemma[0])
    return lemmas

vectorizer = TfidfVectorizer(tokenizer = lemmatize_with_estnltk)

In [ ]:
# Andmete viimine vektorkujule
matrix = vectorizer.fit_transform(column_list)
matrix.shape

In [35]:
#salvestame matrixi maha, et ei peaks iga kord uuesti laadima
#https://stackoverflow.com/questions/8955448/save-load-scipy-sparse-csr-matrix-in-portable-data-format
#from scipy import sparse

#sparse.save_npz("countvec_matrix.npz", matrix)

#uuesti avamiseks
#your_matrix_back = sparse.load_npz("countvec_matrix.npz")

In [ ]:
#loodud proovilause, millega testida sarnasusi
listike=['sisendkäibemaksu mahaarvamine']

In [ ]:
# Proovilause viimine vektorkujule
vordluslist = vectorizer.transform(listike)
vordluslist.shape

In [ ]:
#Leian koossiinuse sarnasused
#Leian esimese 200 sarnase indeksid
#Leian esimese 200 koossiinuse sarnasused, mis vastavad indeksitele
#ühendan indeksid ja koosiinuse sarnasused ühte tabelisse
sarnased_maatriks=cosine_similarity(matrix, vordluslist)
sorteeritud_indeksid=np.argsort(np.squeeze(sarnased_maatriks))
listik_indx=[]
for i in range(1,201):
    listik_indx.append(sorteeritud_indeksid[len(sorteeritud_indeksid) - i])
koos_dist=sarnased_maatriks[listik_indx]
koos_dist_jarj=[]
for i in koos_dist:
    koos_dist_jarj.append(i[0])      
print(koos_dist_jarj)
print(listik_indx)
tabel_sarnasus = pd.DataFrame(columns=['Kohtuasja_indx', 'Koos_sarnasus', 'Jarjekord'])
tabel_sarnasus['Kohtuasja_indx'] = listik_indx
tabel_sarnasus['Koos_sarnasus'] = koos_dist_jarj
tabel_sarnasus['Jarjekord'] = list(range(1, 201))
Tabel_sarnased=tabel_sarnasus.set_index('Kohtuasja_indx') 
#tabel_sarnasus
Tabel_sarnased

In [ ]:
#Filtreerin indeksite põhjal välja sarnased lõigud
filtreeritud_sarnased=data[data.index.isin(listik_indx)]
filtreeritud_sarnased

In [ ]:
# Ühendan kaks tabelit ja filtreerin välja Koos_sarnasus,	Jarjekord, Lõigu sisu, Kohtuasja_nr
Tulemus = pd.concat([Tabel_sarnased, filtreeritud_sarnased], axis=1)
(Tulemus[['Koos_sarnasus', 'Jarjekord','Lõigu sisu', 'Kohtuasja_nr']]).sort_values('Jarjekord', ascending=True)

In [41]:
# Tulemuse salvestamine - 3 tabelit
#Count_vec_tulemus
#Count_vec_tulemus_ainult_reso
#Count_vec_tulemus_ilma_resota
Tulemus.to_csv("Count_vec_tulemus_ilma_resota.csv", index=False)